# Using crack submodels in PyBaMM
In this notebook we show how to use the crack submodel with battery DFN or SPM models. To see all of the models and submodels available in PyBaMM, please take a look at the documentation [here](https://pybamm.readthedocs.io/en/latest/source/models/index.html).

In [1]:
%pip install pybamm -q    # install PyBaMM if it is not installed
import pybamm
import os
import numpy as np
import matplotlib.pyplot as plt
os.chdir(pybamm.__path__[0]+'/..')

Note: you may need to restart the kernel to use updated packages.


Then we load the DFN, SPMe or SPM. When you load a model in PyBaMM it builds by default. Building the model sets all of the model variables and sets up any variables which are coupled between different submodels: this is the process which couples the submodels together and allows one submodel to access variables from another. If you would like to swap out a submodel in an exisitng battery model you need to load it without building it by passing the keyword `build=False`

In [2]:
# model = pybamm.lithium_ion.SPMe(
# model = pybamm.lithium_ion.SPM(
model = pybamm.lithium_ion.DFN(
    build=False,
    options = {
        "particle": "Fickian diffusion",  
        "thermal": "lumped", 
        "particle cracking": "both", # other options are "positive", "negative" or "none"
    }
)

This collects all of the submodels which make up the DFN, but doesn't build the model. 

In [3]:
param = model.default_parameter_values
## Below commands are used to add the mechanical parameters missing from other parameter sets 
import pandas as pd
mechanics = pd.read_csv("pybamm/input/parameters/lithium-ion/mechanicals/lico2_graphite_Ai2020/parameters.csv", 
                        index_col=0, comment="#", skip_blank_lines=True, header=None)[1][1:].dropna().astype(float).to_dict()
param.update(mechanics, check_already_exists=False)
## It can update the speed of crack propagation using the commands below:
# param.update({"Negative electrode Cracking rate":3.9e-20*1000})


We can get the default parameters for the model and update them with the parameters required by the cracking model. Eventually, we would like these to be added to their own chemistry (you might need to adjust the path to the parameters file to your system).
Now the model can be processed and solved in the usual way, and we still have access to model defaults such as the default geometry and default spatial methods

In [4]:
experiment = pybamm.Experiment([
    "Discharge at 1C until 3.2 V",
    "Charge at 1C until 4 V",
])
sim = pybamm.Simulation(
    model,
    experiment=experiment,
    parameter_values=param,
    solver=pybamm.CasadiSolver(dt_max=600),
)
solution = sim.solve()
# plot
quick_plot = pybamm.QuickPlot(solution)
quick_plot.dynamic_plot()

interactive(children=(FloatSlider(value=0.0, description='t', max=2.0595910299911866, step=0.02059591029991186…

Plot the results as required.

In [5]:
# extract voltage
stress_t_n_surf = solution["Negative particle surface tangential stress"]
c_s_n = solution["Negative particle concentration"]
c_s_surf_t=solution["Negative particle surface concentration"]
disp_t = solution["Negative particle surface displacement [m]"]
l_cr_n_t = solution["Negative particle crack length"]
dl_cr = solution ["Negative particle cracking rate"]
t_all = solution["Time [s]"].entries
x = solution["x [m]"].entries[0:19, 0]
c_s_n = solution['Negative particle concentration']
r_n = solution["r_n [m]"].entries[:, 0, 0]

# plot
def plot_concentrations(t):
    f, (ax1, ax2, ax3, ax4) = plt.subplots(1, 4 ,figsize=(20,4))
    ax1.plot(x, stress_t_n_surf(t=t,x=x))
    ax1.set_xlabel(r'$x_n$ [m]')
    ax1.set_ylabel('$\sigma_t/E_n$')
    
    plot_c_n, = ax2.plot(r_n, c_s_n(r=r_n,t=t,x=x[0]))  # can evaluate at arbitrary x (single representative particle)
    ax2.set_ylabel('Negative particle concentration')
    ax2.set_xlabel(r'$r_n$ [m]')
    ax2.set_ylim(0, 1)
    ax2.set_title('Close to current collector')
    ax2.grid()
    
    plot_c_n, = ax3.plot(r_n, c_s_n(r=r_n,t=t,x=x[10]))  # can evaluate at arbitrary x (single representative particle)
    ax3.set_ylabel('Negative particle concentration')
    ax3.set_xlabel(r'$r_n$ [m]')
    ax3.set_ylim(0, 1)  
    ax3.set_title('In the middle')
    ax3.grid()

    plot_c_n, = ax4.plot(r_n, c_s_n(r=r_n,t=t,x=x[-1]))  # can evaluate at arbitrary x (single representative particle)
    ax4.set_ylabel('Negative particle concentration')
    ax4.set_xlabel(r'$r_n$ [m]')
    ax4.set_ylim(0, 1)  
    ax4.set_title('Close to separator')
    ax4.grid()
    plt.show()
    
import ipywidgets as widgets
widgets.interact(plot_concentrations, t=widgets.FloatSlider(min=0,max=7200,step=10,value=0));

interactive(children=(FloatSlider(value=0.0, description='t', max=7200.0, step=10.0), Output()), _dom_classes=…

Plot results using the default functions

In [6]:
label = ["Crack model"]
output_variables = [
    "Negative particle crack length", 
    "Positive particle crack length",
    "X-averaged negative particle crack length",
    "X-averaged positive particle crack length"
]
quick_plot = pybamm.QuickPlot(solution, output_variables, label,variable_limits='tight')
quick_plot.dynamic_plot();


interactive(children=(FloatSlider(value=0.0, description='t', max=2.0595910299911866, step=0.02059591029991186…

## References

[1] M Doyle, TF Fuller and J Newman. "Modeling of galvanostatic charge and discharge of the lithium/polymer/insertion cell." Journal of the Electrochemical Society 140.6 (1993): 1526-1533.

[2] Ai, W., Kraft, L., Sturm, J., Jossen, A., & Wu, B. (2019). Electrochemical Thermal-Mechanical Modelling of Stress Inhomogeneity in Lithium-Ion Pouch Cells. Journal of The Electrochemical Society, 167(1), 013512.